## ADES demonstration - OGC API Processes 

Deploy a CWL application package as a WPS processing service and submit an execution request

In [ ]:
import requests
import os
import getpass
import json
import yaml
from time import sleep

## Discover a registered process

In [ ]:
rm_endpoint = 'https://catalog.terradue.com/eoepca-services/search'

This is the identifier from the previous demonstration notebook:

In [ ]:
application_package_id = 'd5d2d9af95cade45f7602d36ff07aa22'

Query the resource manager catalog to discover the application package:

In [ ]:
payload = {'uid': application_package_id,
           'format': 'json'}

r = requests.get(rm_endpoint, 
                 params=payload)
                 
print(r.status_code, r.reason)

In [ ]:
response = r.json()

response

Here's the Application Package CWL:

In [ ]:
application_package = yaml.safe_load(response['features'][0]['properties']['offering']['content']['#text'])

In [ ]:
application_package

## Deploy the process

In [ ]:
endpoint = 'http://gavi.terradue.com:8888'
workspace = '/terradue'
print(endpoint)

In [ ]:
deploy_headers = {'Content-Type': 'application/json',
                  'Accept': 'application/json', 
                  'Prefer': 'respond-async'} 

In [ ]:
deploy_payload = {'inputs': [{'id': 'applicationPackage',
                              'input': {'format': {'mimeType': 'application/xml'},
                                        'value': {'href': f'https://catalog.terradue.com/eoepca-services/search?uid={application_package_id}'}}}],
                  'outputs': [{'format': {'mimeType': 'string',
                                          'schema': 'string',
                                          'encoding': 'string'},
                               'id': 'deployResult',
                               'transmissionMode': 'value'}],
                  'mode': 'auto',
                  'response': 'raw'}
print(endpoint)

In [ ]:
r = requests.post(f'{endpoint}/{workspace}/wps3/processes',
                  json=deploy_payload,
                  headers=deploy_headers)

print(f'{endpoint}/{endpoint}/{workspace}/wps3/processes')

print(r.status_code, r.reason)

## retrieve GetProcess/processes

In [ ]:
headers = {'Content-Type': 'application/json',
           'Accept': 'application/json'}

In [ ]:
r = requests.get(f'{endpoint}/{workspace}/wps3/processes',
                 headers=headers)

print(r.status_code, r.reason)

In [ ]:
response = r.json()

response

In [ ]:
for process in response:
    
    print('Process id: {}\n{} - {}\n'.format(process['id'], 
                                             process['title'], 
                                             process['abstract']))

## Describe  vegetation-index2 /processes/{id}

In [ ]:
process_id = 'vegetation-index2'

In [ ]:
r = requests.get(f'{endpoint}/{workspace}/wps3/processes/{process_id}',
                 headers=headers)

print(r.status_code, r.reason)

In [ ]:
response = r.json()

response

List the inputs

In [ ]:
inputs = response['process']['inputs']

In [ ]:
inputs

## Execute - /processes/{id}/jobs

In [ ]:
execute_request = {'inputs': [{'id': 'input_reference',
                               'input': {'dataType': {'name': 'string'},
                                         'value': 'https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808'}},
                              {'id': 'aoi',
                               'input': {'dataType': {'name': 'string'},
                                         'value': 'POLYGON((136.112726861895 -36.227897298303,137.333826362695 -36.2103069464338,137.305145407058 -35.2211228310596,136.099040812374 -35.2380875358202,136.112726861895 -36.227897298303))'}}],
                   'outputs': [{'format': {'mimeType': 'string',
                                           'schema': 'string',
                                           'encoding': 'string'},
                                'id': 'wf_outputs',
                                'transmissionMode': 'value'}],
                   'mode': 'async',
                   'response': 'raw'}

In [ ]:
execution_headers = {'Content-Type': 'application/json',
                       'Accept': 'application/json', 
                       'Prefer': 'respond-async'}

In [ ]:
process_id = 'vegetation-index2'

In [ ]:
r = requests.post(f'{endpoint}/{workspace}/wps3/processes/{process_id}/jobs',
                  json=execute_request,
                  headers=execution_headers)

print(r.status_code, r.reason)

In [ ]:
job_location = r.headers['Location']

job_location

## Poll /processes/{id}/jobs/{job_id}

In [ ]:
r = requests.get(f'{endpoint}{job_location}',
                 headers=headers)

print(f'{endpoint}{job_location}')

print(r.status_code, r.reason)

In [ ]:
response = r.json()

response

In [ ]:

while response['status'] == 'running':

    r = requests.get(f'{endpoint}{job_location}',
                 headers=headers)
    
    response = r.json()
    
    if response['status'] == 'failed': 
    
        print(response)
        
        break
        
    if response['status'] == 'successful':  

        print(response['links'][0]['href'])
        
        break
    
    else:
        
        print('Polling - {}'.format(response['status']))
        
        sleep(30)
        

## Results - /processes/{id}/jobs/{job_id}/result

In [ ]:
response

In [ ]:
r = requests.get(f'{endpoint}/{job_location}/result',
                 headers=headers)

print(r.status_code, r.reason)

In [ ]:
response = r.json()

response

In [ ]:
results = json.loads(response['outputs'][0]['value']['inlineValue'])

results

## Inspect the results

Use the STAC catalog endpoint in the "Exploiting the result.ipynb" Notebook 

In [ ]:
stac_catalog_endpoint = results['stac:catalog']['href']

stac_catalog_endpoint